# AX=YB
This notebook shows the process of calibrating the extrinsic parameters between the Vicon marker coordinate frame installed on an object with respect to the object CAD coordinate frame. We use the ICG 3D tracker algorithm to track the 3D pose of the object through its pointcloud observation and the ROS2 vicon bridge to get the pose of the markers installed on it.

## Instantiate the Interfaces
### ROS2

In [ ]:
from SimpleHandEye.interfaces.utils import addFoxyPath
addFoxyPath('/opt/ros/foxy')

In [ ]:
from SimpleHandEye.interfaces.ros2 import ROS2TFInterface
import rclpy
rclpy.init()    
tf_interface = ROS2TFInterface('world', 'base_link')

In [ ]:
tf_interface.getPose()

### ROS1

In [ ]:
from SimpleHandEye.interfaces.utils import addNoeticPath
addNoeticPath('/opt/ros/noetic')

In [ ]:
from SimpleHandEye.interfaces.ros import ROSTFInterface, initRosNode
initRosNode()
tf_interface = ROSTFInterface('vicon/world', 'vicon/wall/wall')

In [ ]:
tf_interface.getPose()

### April-Tag Tracker

In [ ]:
from SimpleHandEye.interfaces.cameras import RealSenseCamera
import cv2

def showImage(color_frame, depth_frame, ir1_frame, ir2_frame):
    cv2.imshow('image', color_frame)
    cv2.waitKey(33)

camera = RealSenseCamera(callback_fn=showImage)

intrinsics_params = camera.getIntrinsics()
K = intrinsics_params['RGB']['K']
D = intrinsics_params['RGB']['D']

In [ ]:
from SimpleHandEye.interfaces.apriltag import ApriltagTracker
tracker = ApriltagTracker(tag_size=0.172,
                          intrinsic_matrix=K,
                          distortion_coeffs=D)

tracker.getPose(camera.color_frame, tag_id=0)

In [ ]:
import time
from SimpleHandEye.interfaces.ros import ROSTFPublisher
tracker_tf_publisher = ROSTFPublisher('camera', 'tag0')

for _ in range(100):
    T = tracker.getPose(camera.color_frame, tag_id=0)
    if T is not None:
        tracker_tf_publisher.publish(T)
        time.sleep(0.3) 

## Instantiate the Solver

In [ ]:
from SimpleHandEye.solvers import OpenCVSolver
solver = OpenCVSolver()

## Collect The Calibration Dataset

In [ ]:
import ipywidgets as widgets
import numpy as np
from IPython.display import display
from pprint import pprint
from IPython.display import clear_output
np.set_printoptions(suppress=True, precision=3)

# The dataset
A_list = []
B_list = []
apriltag_info = []
apriltag_imgs_raw = []
apriltag_imgs_udist = []
def on_sample_clicked(b):
    A = tf_interface.getPose()
    img = camera.color_frame
    info = tracker.getPoseAndCorners(img, tag_id=0)
    B = info['pose']
    apriltag_info.append(info)
    apriltag_imgs_raw.append(img)
    apriltag_imgs_udist.append(tracker.undistortImage(img))
    print("A=")
    pprint(A)
    print("B=")
    pprint(B)
    # if A is not None and B is not None:
    A_list.append(A)
    B_list.append(B)
    print("*************")

def on_compute_clicked(b):
    try:
        X,Y = solver.solve(A_list, B_list)
        clear_output(wait=True)
        print("X=")
        pprint(X)
        print("Y=")
        pprint(Y)
    except:
        print("Bad dataset, please record again")
        A_list.clear()
        B_list.clear()
        

sample_button = widgets.Button(description="Sample")
compute_button = widgets.Button(description="Compute")

sample_button.on_click(on_sample_clicked)
compute_button.on_click(on_compute_clicked)
display(sample_button)
display(compute_button)

In [ ]:
X, Y = solver.solve(A_list, B_list)

In [ ]:
camera.close()

In [ ]:
import pickle
with open('dataset.pkl', 'wb') as f:
    data = {
        'A_list': A_list,
        'B_list': B_list,
        'apriltag_info': apriltag_info,
        'apriltag_imgs_raw': apriltag_imgs_raw,
        'apriltag_imgs_udist': apriltag_imgs_udist,
        'camera_matrix': K, 
        'distortion_coeffs': D,
        'X': X,
        'Y': Y
    }
    pickle.dump(data, f)

## Export the Result

In [ ]:
wall_T_tag = X
world_T_camera = Y

In [ ]:
from SimpleHandEye.exporters import export2ROS
export2ROS(wall_T_tag, 'vicon/wall/wall', 'tag', 'wall_T_tag_publisher.launch')
export2ROS(world_T_camera, 'vicon/world', 'camera', 'world_T_camera_publisher.launch')